In [1]:
import sys
import os

import torch

sys.path.append(os.path.abspath("/Users/liuzhe/github/DPL_study/pys"))
from single_env_from_csv import *


timeline: 31, net: 200810.0, cash: 199990, profile: 820.0, direction: 1, in_price: 6894.0, now_price: 6976.0
got here 2


In [2]:
#e = SingleEnv('/Users/liuzhe/github/DPL_study/data/snap.csv', 'rb')
one_long = torch.tensor([1, 0, 0, 0])
one_short = torch.tensor([0, 1, 0, 0])
one_close = torch.tensor([0, 0, 1, 0])
one_hold = torch.tensor([0, 0, 0, 1])





In [3]:
e.render()
e.data

timeline: 31, net: 200810.0, cash: 199990, profile: 820.0, direction: 1, in_price: 6894.0, now_price: 6976.0


,day,time,price
0,20200203,09:00:00,6895.0
1,20200203,09:00:01,6894.0
2,20200203,09:00:02,6886.0
3,20200203,09:00:03,6894.0
4,20200203,09:00:04,6888.0
...,...,...,...
17064,20200203,15:14:35,-1.0
17065,20200203,15:14:36,-1.0
17066,20200203,15:14:37,-1.0
17067,20200203,15:14:38,-1.0


In [4]:
import torch
import torch.nn.functional as F
d1 = torch.nn.Linear(100, 50) # y = A(100x50) * x + b,
input = torch.rand(140, 100)
out = d1(input)
out.shape


d2 = torch.nn.Linear(4, 5)
input_matrix = torch.tensor([[1.0, 0.0, 0.0, 0.0]])
F.relu(d2(input_matrix))

d3 = torch.nn.Conv1d(4, 1, 3)

time_input = torch.tensor([[
    [323.0, 324.0, 323.0, 325.0,], # tick
    [310.0, 310.0, 310.0, 311.0,], # min
    [300.0, 300.0, 300.0, 300.0],  # hour
    [300.0, 300.0, 300.0, 300.0],  # hour
]])

d4 = torch.nn.Conv2d(4, 1, 3)

d3(time_input)


[W NNPACK.cpp:79] Could not initialize NNPACK! Reason: Unsupported hardware.


tensor([[[194.5166, 194.1873]]], grad_fn=<SqueezeBackward1>)

In [86]:
metric_num = len(['o', 'h', 'l', 'c', 'v_delta',])
step_num = 20
time_scales = len(['tick', 'min', '5min', '15min', 'hour', 'day'])
action_count = len(['b1', 's1', 'h', 'c'])

fake_tick = torch.randn((time_scales, step_num, metric_num)) #
fake_tick



tensor([[[-3.0948e-01,  1.0656e+00,  3.9142e-01, -2.1842e-01, -4.5702e-01],
         [-8.1276e-01,  1.7447e-01,  5.0249e-01, -3.5413e-01,  3.5017e-02],
         [ 6.3394e-02,  1.1194e+00, -8.6499e-01,  7.7794e-01, -9.7448e-01],
         [-2.5633e-01,  1.0592e+00,  4.3715e-01,  1.2339e-01,  4.6911e-02],
         [-2.2362e+00, -2.6185e-01,  5.4983e-01, -9.0113e-02,  1.0959e+00],
         [-3.6545e-01, -2.1616e+00, -1.2328e+00, -5.0122e-01,  6.8461e-01],
         [ 3.3591e-01,  8.4470e-01, -1.1091e-01,  6.9038e-01, -2.8962e-01],
         [ 4.4338e-01,  8.1973e-01, -3.2848e-01, -1.3372e+00,  1.1463e+00],
         [ 2.8764e-01, -1.4550e-01, -3.8360e-01,  9.4777e-01,  1.1928e+00],
         [ 9.7568e-01,  2.8647e-01, -4.3111e-01, -1.4688e+00, -1.0558e+00],
         [-2.9751e-01, -8.6467e-02, -1.6863e+00,  1.8837e-01, -1.2697e+00],
         [ 3.7945e-01, -1.1042e+00, -3.8351e-02,  2.5243e-01, -3.9608e-01],
         [ 3.8342e-02,  2.5415e+00,  2.1143e-01, -2.7910e-01, -1.7841e-01],
         [ 8

In [16]:
conv_out_rows = 10

mod = torch.nn.Sequential(
    torch.nn.Conv1d(step_num, conv_out_rows, 4),
    torch.nn.Flatten(end_dim=1)
)
out = mod(fake_tick)
out.size()

torch.Size([6, 10, 2])

In [58]:
out.reshape(-1).size()

fake_tick.size()
t1 = torch.nn.Conv2d(time_scales, 9, (5, 3))
t1(fake_tick.unsqueeze(0)).view(-1).size()


torch.Size([432])

In [87]:
kernel_h = 5 # segment_range
kernel_w = 3 # metric_kinds

cons = torch.nn.Sequential(
    torch.nn.Conv2d(time_scales, time_scales, (kernel_h, kernel_w), padding=1),
    torch.nn.ReLU(),
    torch.nn.Conv2d(time_scales, time_scales, (kernel_h, kernel_w), padding=1),
    torch.nn.ReLU(),
    torch.nn.Conv2d(time_scales, time_scales, (kernel_h, kernel_w), padding=1),
    torch.nn.ReLU()
)

cons(fake_tick.unsqueeze(0)).size()
#fake_tick.unsqueeze(0).size()
in_w = len(cons(fake_tick.unsqueeze(0)).view(-1))

lins = torch.nn.Sequential(
    torch.nn.Linear(in_w, in_w),
    torch.nn.Linear(in_w, in_w),
    torch.nn.Linear(in_w, action_count),
)

def forward(x):
    con_out = cons(x.unsqueeze(0))
    lin_out = lins(con_out.view(-1))
    #return torch.nn.functional.softmax(lin_out)
    return lin_out

forward(fake_tick)

/var/folders/zp/br0t0rj942x005pnp5ghxfy40000gn/T/ipykernel_90388/2052592867.py:26: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return torch.nn.functional.softmax(lin_out)


tensor([0.2513, 0.2474, 0.2508, 0.2505], grad_fn=<SoftmaxBackward>)